In [211]:
import openmeteo_requests
import requests_cache
import pandas as pd
import dates_to_call, gps_coords_to_call
from pandas.core.interchange import dataframe
from retry_requests import retry


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
open_meteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
def call_for_data(latitude, longitude, start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
		"latitude": latitude,
		"longitude": longitude,
		"start_date": start_date,
		"end_date": end_date,
		"hourly": ["temperature_2m", "precipitation", "cloud_cover", "wind_speed_100m"],
		"timezone": "auto",
        "temperature_unit": "fahrenheit"
	}
    responses = open_meteo.weather_api(url, params=params)
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    return response

In [212]:
test_start_date = '1960-06-17'
test_end_date = '1960-06-30'
test_latitude = '39.28'
test_longitude = '-76.60'
test_unit = 'fahrenheit'
test_response = call_for_data(test_latitude, test_longitude, test_start_date, test_end_date)

In [213]:
test_response

In [214]:
def make_dataframe(api_response):
    """
    takes the result of call for data and transforms it into a dataframe
    :param api_response:
    :return:
    """
    hourly = test_response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(2).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(3).ValuesAsNumpy()
    hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s"),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m

    return pd.DataFrame(data = hourly_data)


In [215]:
make_dataframe(test_response)

,date,temperature_2m,precipitation,cloud_cover,wind_speed_100m
0,1960-06-17 05:00:00,71.058197,0.0,9.0,32.072342
1,1960-06-17 06:00:00,71.238197,0.0,4.0,31.731092
2,1960-06-17 07:00:00,71.148201,0.0,4.0,30.911821
3,1960-06-17 08:00:00,71.148201,0.0,0.0,29.957972
4,1960-06-17 09:00:00,71.058197,0.0,0.0,29.284180
...,...,...,...,...,...
331,1960-07-01 00:00:00,82.488197,0.0,82.0,13.910169
332,1960-07-01 01:00:00,78.798203,0.0,97.0,16.595179
333,1960-07-01 02:00:00,76.008202,0.0,100.0,19.201874
334,1960-07-01 03:00:00,73.578201,0.0,99.0,19.813087


In [216]:
# #function:
# input start/end/lat/long/unit
# define empty dataframe
# for loop:
# 	call to get 1 year
# 	enter year to dataframe
# return full dataframe
is_first_result = True
#lat = [0], long = [1]
location = gps_coords_to_call.convert_location_to_gps("baltimore")
#replace with variable
years_to_call = dates_to_call.generate_master_date_list(6, 23)
for year in years_to_call:
    year_data = call_for_data(location[0], location[1], year[1], year[0])
    if is_first_result:
        master_dataframe = make_dataframe(year_data)
        is_first_result = False
    else:
        year_dataframe = make_dataframe(year_data)
        master_dataframe = pd.concat([master_dataframe, year_dataframe])
        year_dataframe = None


In [217]:
master_dataframe

,date,temperature_2m,precipitation,cloud_cover,wind_speed_100m
0,1960-06-17 05:00:00,71.058197,0.0,9.0,32.072342
1,1960-06-17 06:00:00,71.238197,0.0,4.0,31.731092
2,1960-06-17 07:00:00,71.148201,0.0,4.0,30.911821
3,1960-06-17 08:00:00,71.148201,0.0,0.0,29.957972
4,1960-06-17 09:00:00,71.058197,0.0,0.0,29.284180
...,...,...,...,...,...
331,1960-07-01 00:00:00,82.488197,0.0,82.0,13.910169
332,1960-07-01 01:00:00,78.798203,0.0,97.0,16.595179
333,1960-07-01 02:00:00,76.008202,0.0,100.0,19.201874
334,1960-07-01 03:00:00,73.578201,0.0,99.0,19.813087
